In [2]:
from typing import Tuple

import itertools
import logging

import numpy
import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

In [3]:
logger = logging.getLogger('cmdstanpy')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('prophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

logger = logging.getLogger('fbprophet')
logger.addHandler(logging.NullHandler())
logger.propagate = False
logger.setLevel(logging.CRITICAL)

In [4]:
country = 'france'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [4]:
def make_dataset(df_processed: pandas.DataFrame) -> pandas.DataFrame:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)

    # unemployment rate, being a percentage, can only be between 0 to 100
    df_mrd['floor'] = 0
    df_mrd['cap'] = 100
    return df_mrd

In [5]:
df_mrd = make_dataset(df)

In [6]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [7]:
df_train, df_test = train_test_split(df_mrd, 12)

In [8]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse'):
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # other params
    model_params = {
        'growth': 'logistic', 'seasonality_mode': 'additive',
        'daily_seasonality': False, 'weekly_seasonality': False, 'yearly_seasonality': False
    }
    cv_params = {'horizon': '30 days', 'parallel': 'processes'}
    pm_params = {'metrics': [eval_metric, 'coverage'], 'rolling_window': 0.1, 'monthly': True}

    print(f'Total hyper-parameter set count: {len(all_params)}')

    # Use cross validation to evaluate all parameters
    iteration_count: int = 1
    for hyper_params in all_params:
        print(f'Set: {iteration_count}')
        print(f'{hyper_params}')

        m = Prophet(**hyper_params, **model_params).add_seasonality(
            name='quarterly', period=365.25/4, fourier_order = 5
            ).fit(df)
        df_cv = cross_validation(m, **cv_params)
        df_p = performance_metrics(df_cv, **pm_params)
        eval_metric_values.append(df_p[eval_metric].values[0])

        print(f'{eval_metric.upper()}: {df_p[eval_metric].values[0]}')
        iteration_count += 1
        print()

    best_params: dict = all_params[numpy.argmin(eval_metric_values)]

    print('Training model on the best hyper-parameter set.')
    print(f'{best_params}')
    
    auto_model: Prophet = Prophet(**best_params, **model_params).add_seasonality(
        name='quarterly', period=365.25/4, fourier_order = 5
        ).fit(df)

    print('Cross-Validating best model.')
    auto_model_cv = cross_validation(auto_model, **cv_params)
    auto_model_p: pandas.DataFrame = performance_metrics(auto_model_cv, **pm_params)

    return auto_model, auto_model_p, best_params

In [9]:
param_grid = {  
    'changepoint_prior_scale': [0.5, 0.75],
    'seasonality_prior_scale': [0.1, 1.0],
}

auto_model, auto_model_p, best_params = train_model(df_train, param_grid)

Total hyper-parameter set count: 4
Set: 1
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 0.5574878180527727

Set: 2
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 1.0}
RMSE: 0.8187668349288306

Set: 3
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 0.1}


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

RMSE: 0.5584820039483905

Set: 4
{'changepoint_prior_scale': 0.75, 'seasonality_prior_scale': 1.0}
RMSE: 0.8249915973148649

Training model on the best hyper-parameter set.
{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}
Cross-Validating best model.


/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1294: RuntimeWarning: invalid value encountered in add
  m_t[indx] += gammas[s]
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/prophet/forecaster.py:1286: RuntimeWarning: divide by zero encountered in scalar divide
  * (1 - k_cum[i] / k_cum[i + 1])  # noqa W503
/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwar

In [10]:
best_params

{'changepoint_prior_scale': 0.5, 'seasonality_prior_scale': 0.1}

In [11]:
auto_model_p

,horizon,rmse,coverage
0,1,0.557488,0.274834
1,2,0.554581,0.289681


In [12]:
def test_model(df_test: pandas.DataFrame, model):
    "return predicted values and rmse"
    df_predicted = model.predict(df_test)
    rmse = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [13]:
df_predicted, rmse = test_model(df_test, auto_model)

In [14]:
df_predicted

,ds,trend,cap,floor,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,quarterly,quarterly_lower,quarterly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,12.722389,100,0,7.175624,7.887138,12.722389,12.722389,-5.182596,-5.182596,-5.182596,-5.182596,-5.182596,-5.182596,0.0,0.0,0.0,7.539794
1,2022-04-01,12.691848,100,0,7.142157,7.849854,12.692060,12.692060,-5.199765,-5.199765,-5.199765,-5.199765,-5.199765,-5.199765,0.0,0.0,0.0,7.492083
2,2022-05-01,12.662352,100,0,7.126343,7.860319,12.662958,12.662958,-5.175253,-5.175253,-5.175253,-5.175253,-5.175253,-5.175253,0.0,0.0,0.0,7.487098
3,2022-06-01,12.631934,100,0,7.033614,7.734053,12.632976,12.632976,-5.244040,-5.244040,-5.244040,-5.244040,-5.244040,-5.244040,0.0,0.0,0.0,7.387894
4,2022-07-01,12.602557,100,0,7.068032,7.839095,12.597846,12.612532,-5.154185,-5.154185,-5.154185,-5.154185,-5.154185,-5.154185,0.0,0.0,0.0,7.448372
5,2022-08-01,12.572262,100,0,6.991480,7.721675,12.555733,12.603140,-5.209964,-5.209964,-5.209964,-5.209964,-5.209964,-5.209964,0.0,0.0,0.0,7.362298
6,2022-09-01,12.542029,100,0,7.014577,7.777318,12.504367,12.596008,-5.151828,-5.151828,-5.151828,-5.151828,-5.151828,-5.151828,0.0,0.0,0.0,7.390201
7,2022-10-01,12.512831,100,0,6.922777,7.678619,12.450444,12.596631,-5.222495,-5.222495,-5.222495,-5.222495,-5.222495,-5.222495,0.0,0.0,0.0,7.290336
8,2022-11-01,12.482721,100,0,6.885396,7.691582,12.389541,12.595966,-5.201764,-5.201764,-5.201764,-5.201764,-5.201764,-5.201764,0.0,0.0,0.0,7.280957
9,2022-12-01,12.453642,100,0,6.823617,7.657757,12.332520,12.594767,-5.212553,-5.212553,-5.212553,-5.212553,-5.212553,-5.212553,0.0,0.0,0.0,7.241089


In [15]:
rmse

0.14810912558555822

In [1]:
with open(f'{country}_prophet_cv_model_2.json', 'w') as f:
    f.write(model_to_json(auto_model))

NameError: name 'country' is not defined

In [ ]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [ ]:
df_future_prediction = auto_model.predict(df_future)

In [ ]:
df_future_prediction